In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, recall_score, f1_score, roc_auc_score

from shared.utils import load_data
from datasets import preprocess_dataset
from intrusion_detection_systems.models import cnn_model, mlp_model, rnn_model
from intrusion_detection_systems import train_dl_model, evaluate_dl_model
from agent.DQN_ML_Pool import DQNModelSelector
from agent.MAB_ML_Pool import MultiArmedBanditThompsonSampling

from art.estimators.classification.scikitlearn import ScikitlearnClassifier
from art.utils import to_categorical
from art.attacks.evasion import ZooAttack
from art.estimators.classification import SklearnClassifier
from art.estimators.classification import PyTorchClassifier

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
name = "CIC-IDS_2017_2"
df = load_data(
            [
                "./shared/data/CIC_2017/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv",
                # "./shared/data/CIC_2017/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv",
                # "./shared/data/CIC_2017/Friday-WorkingHours-Morning.pcap_ISCX.csv",
                # "./shared/data/CIC_2017/Monday-WorkingHours.pcap_ISCX.csv",
                # "./shared/data/CIC_2017/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv"
                # "./shared/data/CIC_2017/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv",
                # "./shared/data/CIC_2017/Tuesday-WorkingHours.pcap_ISCX.csv"
            ],
            42
        )
print("Dataset loaded")
df_preprocessed = preprocess_dataset(
    df, save=True, dataset_type="CIC_2017", seed=42, load=False, name_save=name, name_load=name)
print("Dataset preprocessed")

X_train_full, y_train_full = df_preprocessed.x_train, df_preprocessed.y_train
X_test_full, y_test_full = df_preprocessed.x_test, df_preprocessed.y_test
print(f"Train full shape: {X_train_full.shape}, labels distribution: {np.unique(y_train_full, return_counts=True)}")
print(f"Test full shape: {X_test_full.shape}, labels distribution: {np.unique(y_test_full, return_counts=True)}")
y_train_full = np.array([int(str(x).strip()) for x in y_train_full])
y_test_full = np.array([int(str(x).strip()) for x in y_test_full])


In [ ]:
# Train a NB Classifier
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb.fit(X_train_full, y_train_full)
# Train a RF Classifier
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train_full, y_train_full)
# Train a DT Classifier
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(X_train_full, y_train_full)
# Train a LR Classifier
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train_full, y_train_full)

In [ ]:
input_dim = X_train_full.shape[1]
#MLP Model
mlp_model_pt = mlp_model(input_dim)
criterion_mlp_model_pt = nn.CrossEntropyLoss()
optimizer_mlp_model_pt = optim.Adam(mlp_model_pt.parameters(), lr=0.001)

print("--- MLP Model ---")
print(mlp_model_pt)
print(f"Criterion: {criterion_mlp_model_pt}")
print(f"Optimizer: {optimizer_mlp_model_pt}")
# CNN Model
cnn_model_pt = cnn_model(input_dim)
criterion_cnn_model_pt = nn.CrossEntropyLoss()
optimizer_cnn_model_pt = optim.Adam(cnn_model_pt.parameters(), lr=0.001)

print("--- CNN Model ---")
print(cnn_model_pt)
print(f"Criterion: {criterion_cnn_model_pt}")
print(f"Optimizer: {optimizer_cnn_model_pt}")
# RNN Model
rnn_model_pt = rnn_model(input_dim) 
criterion_rnn_model_pt = nn.CrossEntropyLoss()
optimizer_rnn_model_pt = optim.Adam(rnn_model_pt.parameters(), lr=0.001)

print("--- RNN Model (LSTM) ---")
print(rnn_model_pt)
print(f"Criterion: {criterion_rnn_model_pt}")
print(f"Optimizer: {optimizer_rnn_model_pt}")

In [6]:
#Train DL models for attack
# Chia tập train -> train và validation
X_small = X_train_full[:1000]
y_small = y_train_full[:1000]
X_train, X_val, y_train, y_val = train_test_split(
    X_small, y_small, test_size=0.2, random_state=42, stratify=y_small
)

# Chuyển sang tensor
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)

X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)

X_test_tensor = torch.tensor(X_test_full[:1000], dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_full[:1000], dtype=torch.long)

# Dataloader
batch_size = 128
train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(TensorDataset(X_val_tensor, y_val_tensor), batch_size=batch_size, shuffle=False)
test_loader  = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=batch_size, shuffle=False)


In [7]:
#MLP Target Model 
mlp_model_target = mlp_model(input_dim)
criterion_mlp_model_target = nn.CrossEntropyLoss()
optimizer_mlp_model_target = optim.Adam(mlp_model_target.parameters(), lr=0.001)

# CNN Target Model
cnn_model_target = cnn_model(input_dim)
criterion_cnn_model_target = nn.CrossEntropyLoss()
optimizer_cnn_model_target = optim.Adam(cnn_model_target.parameters(), lr=0.001)

# RNN Target Model
rnn_model_target = rnn_model(input_dim) 
criterion_rnn_model_target = nn.CrossEntropyLoss()
optimizer_rnn_model_target = optim.Adam(rnn_model_pt.parameters(), lr=0.001)

# ----- MLP -----
mlp_target = train_dl_model(mlp_model_target, train_loader, val_loader, criterion_mlp_model_target, optimizer_mlp_model_target, device, num_epochs=50)
evaluate_dl_model(mlp_target, test_loader, device,model_type='mlp', name="Test")

# ----- CNN -----
cnn_target = train_dl_model(cnn_model_target, train_loader, val_loader, criterion_cnn_model_target, optimizer_cnn_model_target, device,num_epochs=50)
evaluate_dl_model(cnn_target, test_loader, device,model_type='cnn', name="Test")

# ----- RNN -----
rnn_target = train_dl_model(rnn_model_target, train_loader, val_loader, criterion_rnn_model_target, optimizer_rnn_model_target, device,num_epochs=50)
evaluate_dl_model(rnn_target, test_loader, device,model_type='rnn', name="Test")


--- Training MLPModel ---
Epoch [1/50], Train Loss: 0.6914, Train Acc: 0.5825, Val Loss: 0.6807, Val Acc: 0.7350
Epoch [2/50], Train Loss: 0.6781, Train Acc: 0.7025, Val Loss: 0.6661, Val Acc: 0.7600
Epoch [3/50], Train Loss: 0.6632, Train Acc: 0.7275, Val Loss: 0.6463, Val Acc: 0.7700
Epoch [4/50], Train Loss: 0.6430, Train Acc: 0.7338, Val Loss: 0.6165, Val Acc: 0.7750
Epoch [5/50], Train Loss: 0.6138, Train Acc: 0.7388, Val Loss: 0.5760, Val Acc: 0.7750
Epoch [6/50], Train Loss: 0.5760, Train Acc: 0.7388, Val Loss: 0.5251, Val Acc: 0.7850
Epoch [7/50], Train Loss: 0.5288, Train Acc: 0.7512, Val Loss: 0.4677, Val Acc: 0.8100
Epoch [8/50], Train Loss: 0.4782, Train Acc: 0.7738, Val Loss: 0.4099, Val Acc: 0.8300
Epoch [9/50], Train Loss: 0.4257, Train Acc: 0.7837, Val Loss: 0.3548, Val Acc: 0.8400
Epoch [10/50], Train Loss: 0.3743, Train Acc: 0.8512, Val Loss: 0.3025, Val Acc: 0.9550
Epoch [11/50], Train Loss: 0.3207, Train Acc: 0.9413, Val Loss: 0.2508, Val Acc: 0.9600
Epoch [12/50],

In [8]:
SUB_TRAIN = min(1000, len(X_train_full))
SUB_TEST = min(2000, len(X_test_full))
# Sử dụng mẫu nhỏ hoặc toàn bộ dữ liệu
X_train, y_train = X_train_full[:SUB_TRAIN], y_train_full[:SUB_TRAIN]
X_test, y_test = X_test_full[:SUB_TEST], y_test_full[:SUB_TEST]
# Nếu muốn dùng toàn bộ dữ liệu, bỏ dòng trên và dùng:
# X_train, y_train = X_train_full, y_train_full
# X_test, y_test = X_test_full, y_test_full
print("📌 Nhãn của tập huấn luyện (y_train):", np.unique(y_train, return_counts=True))
print("📌 Nhãn của tập kiểm thử (y_test):", np.unique(y_test, return_counts=True))

📌 Nhãn của tập huấn luyện (y_train): (array([0, 1]), array([423, 577], dtype=int64))
📌 Nhãn của tập kiểm thử (y_test): (array([0, 1]), array([ 855, 1145], dtype=int64))


In [9]:
# =====================
# DQN Training and Evaluation
# =====================
models = [
    RandomForestClassifier(random_state=42),
    DecisionTreeClassifier(random_state=42),
    GaussianNB()
]

dqn = DQNModelSelector(models, n_clusters=3)

print("Training DQN-based selector with cluster-specific DQNs...")
dqn.train(X_train, y_train)

print("Predicting on test set...")
y_pred_dqn, arms = dqn.predict(X_test)

if isinstance(y_test, pd.DataFrame):
    if "Label" in y_test.columns:
        y_test = y_test["Label"].values
    else:
        y_test = y_test.iloc[:, 0].values
elif isinstance(y_test, pd.Series):
    y_test = y_test.values

y_test = np.array([int(str(x).strip()) for x in y_test])
y_pred_dqn = np.array([int(x) for x in y_pred_dqn])

print(classification_report(y_test, y_pred_dqn))
print("Accuracy:", accuracy_score(y_test, y_pred_dqn))
print("Recall (macro):", recall_score(y_test, y_pred_dqn, average='macro'))
print("F1 Score (macro):", f1_score(y_test, y_pred_dqn, average='macro'))
print("ROC AUC:", roc_auc_score(y_test, y_pred_dqn))

Training DQN-based selector with cluster-specific DQNs...


d:\anaconda3\envs\py39DQN\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(



 Training models for cluster 0
Cluster 0, Epoch 1/30, Avg Reward: 1.0022
Cluster 0, Epoch 2/30, Avg Reward: 0.9962
Cluster 0, Epoch 3/30, Avg Reward: 1.0050
Cluster 0, Epoch 4/30, Avg Reward: 1.0023
Cluster 0, Epoch 5/30, Avg Reward: 1.0059
Cluster 0, Epoch 6/30, Avg Reward: 0.9959
Cluster 0, Epoch 7/30, Avg Reward: 0.9930
Cluster 0, Epoch 8/30, Avg Reward: 1.0055
Cluster 0, Epoch 9/30, Avg Reward: 1.0047
Cluster 0, Epoch 10/30, Avg Reward: 1.0056
Cluster 0, Epoch 11/30, Avg Reward: 0.9896
Cluster 0, Epoch 12/30, Avg Reward: 1.0039
Cluster 0, Epoch 13/30, Avg Reward: 0.9961
Cluster 0, Epoch 14/30, Avg Reward: 1.0037
Cluster 0, Epoch 15/30, Avg Reward: 0.9902
Cluster 0, Epoch 16/30, Avg Reward: 0.9995
Cluster 0, Epoch 17/30, Avg Reward: 0.9995
Cluster 0, Epoch 18/30, Avg Reward: 1.0067
Cluster 0, Epoch 19/30, Avg Reward: 1.0027
Cluster 0, Epoch 20/30, Avg Reward: 0.9993
Cluster 0, Epoch 21/30, Avg Reward: 1.0085
Cluster 0, Epoch 22/30, Avg Reward: 1.0047
Cluster 0, Epoch 23/30, Avg Rew

In [10]:
# =====================
# MAB Training and Evaluation
# =====================
mab = MultiArmedBanditThompsonSampling(n_arms=3, n_clusters=2)
mab.train(X_train, y_train)

y_pred_mab, selected_arms = mab.predict(X_test)

if isinstance(y_test, pd.DataFrame):
    if "Label" in y_test.columns:
        y_test = y_test["Label"].values
    else:
        y_test = y_test.iloc[:, 0].values
elif isinstance(y_test, pd.Series):
    y_test = y_test.values

y_test = np.array([int(str(x).strip()) for x in y_test])
y_pred_mab = np.array([int(x) for x in y_pred_mab])

print(classification_report(y_test, y_pred_mab))
print("Accuracy:", accuracy_score(y_test, y_pred_mab))
print("Recall (macro):", recall_score(y_test, y_pred_mab, average='macro'))
print("F1 Score (macro):", f1_score(y_test, y_pred_mab, average='macro'))
print("ROC AUC:", roc_auc_score(y_test, y_pred_mab))

d:\anaconda3\envs\py39DQN\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Cluster 0: 287 samples
Training arm 0 on cluster 0
cluster_X_train shape: (287, 68)
cluster_y_train shape: (287,)
arm_mask shape: (287,)
Training arm 1 on cluster 0
cluster_X_train shape: (287, 68)
cluster_y_train shape: (287,)
arm_mask shape: (287,)
Training arm 2 on cluster 0
cluster_X_train shape: (287, 68)
cluster_y_train shape: (287,)
arm_mask shape: (287,)
Cluster 1: 713 samples
Training arm 0 on cluster 1
cluster_X_train shape: (713, 68)
cluster_y_train shape: (713,)
arm_mask shape: (713,)
Training arm 1 on cluster 1
cluster_X_train shape: (713, 68)
cluster_y_train shape: (713,)
arm_mask shape: (713,)
Training arm 2 on cluster 1
cluster_X_train shape: (713, 68)
cluster_y_train shape: (713,)
arm_mask shape: (713,)
Setting reward_sums arm 0 on cluster 0
Setting reward_sums arm 1 on cluster 0
Setting reward_sums arm 2 on cluster 0
Setting reward_sums arm 0 on cluster 1
Setting reward_sums arm 1 on cluster 1
Setting reward_sums arm 2 on cluster 1
              precision    recall  f

In [11]:
attacks_data = X_test_full[y_test_full == 1]
normal_data = X_test_full[y_test_full == 0]
print(attacks_data.shape, normal_data.shape)

FUNCTIONAL_FEATURES = [
 ' min_seg_size_forward',' Bwd Header Length',' Destination Port'
 'Init_Win_bytes_forward',' Init_Win_bytes_backward',' Bwd Packets/s'
 'Total Length of Fwd Packets',' Subflow Fwd Bytes',' Max Packet Length'
 'Bwd Packet Length Max',' Avg Bwd Segment Size',' Bwd Packet Length Mean'
 ' Fwd Packet Length Max',' Average Packet Size',' Packet Length Std'
 ' Packet Length Mean',' Bwd Packet Length Std',' Bwd Packet Length Min'
 ' Fwd Packet Length Std',' Fwd Packet Length Min',' Min Packet Length'
 ' Fwd Packet Length Mean',' Avg Fwd Segment Size',' act_data_pkt_fwd'
 ' Total Fwd Packets','Subflow Fwd Packets',' Total Backward Packets']
print(len(FUNCTIONAL_FEATURES))
FUNCTIONAL_FEATURES_IDEXES = [df.columns.get_loc(c) for c in df.columns if c not in FUNCTIONAL_FEATURES][:-1]
print(FUNCTIONAL_FEATURES_IDEXES, len(FUNCTIONAL_FEATURES_IDEXES))
NON_FUNCTIONAL_FEATURES_IDEXES = [df.columns.get_loc(c) for c in df.columns if c in FUNCTIONAL_FEATURES]
print(NON_FUNCTIONAL_FEATURES_IDEXES, len(NON_FUNCTIONAL_FEATURES_IDEXES))
print(len(df.columns))

(38408, 68) (29316, 68)
19
[0, 1, 2, 4, 5, 6, 8, 9, 10, 11, 12, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 51, 54, 55, 56, 58, 60, 61, 62, 63, 64, 65, 66] 56
[3, 7, 13, 32, 48, 49, 50, 52, 53, 57, 59] 11
68


In [12]:
# Create ART classifier
class Classifier(ScikitlearnClassifier):
    
    def __init__(self, model, clip_values=None, preprocessing=(0, 1), attacks=[]):
        super(Classifier, self).__init__(model=model, clip_values=clip_values, preprocessing=preprocessing)
        self._attacks = attacks

    def predict(self, x, **kwargs):
        # Set attacks features to X
        for i in FUNCTIONAL_FEATURES_IDEXES:
            for j in range(len(x)):
                x[j][i] = self._attacks[j][i]
        predictions = self._model.predict(x)
        return to_categorical(predictions, nb_classes=self._get_nb_classes())

In [13]:
# ===================== Adversarial Attacks on ML Models =====================
models = [nb, dt, lr, rf]
accuracys, drs, f1s, rocs = [], [], [], []
mab_accuracys, mab_drs, mab_f1s, mab_rocs = [], [], [], []

for model in models:
    classifier = SklearnClassifier(model=model, clip_values=(0, 1))
    attack = ZooAttack(classifier=classifier, targeted=False, nb_parallel=60)
    
    x_test_adv = attack.generate(attacks_data[:500], np.zeros((attacks_data[:500].shape[0], 1)))

    non_adv_x_test = np.concatenate((attacks_data[:500], normal_data[:500]))
    non_adv_y_test = np.concatenate((np.ones((attacks_data[:500].shape[0], 1)), np.zeros((normal_data[:500].shape[0], 1))))
    adv_x_test = np.concatenate((x_test_adv, normal_data[:500]))
    adv_y_test = np.concatenate((np.ones((x_test_adv.shape[0], 1)), np.zeros((normal_data[:500].shape[0], 1))))

    print("====================> Model: ", model)
    print("---------- Adversarial data")

    # Predict cho dqn
    y_true = adv_y_test.astype(int).ravel()
    y_pred_dqn = dqn.predict(adv_x_test)[0].astype(int).ravel()

    accuracy = accuracy_score(y_true, y_pred_dqn)
    recall = recall_score(y_true, y_pred_dqn)
    f1 = f1_score(y_true, y_pred_dqn)
    roc = roc_auc_score(y_true, y_pred_dqn)

    accuracys.append(accuracy)
    drs.append(recall)
    f1s.append(f1)
    rocs.append(roc)

    print("===> DQN:")
    print(classification_report(y_true, y_pred_dqn))
    print("Accuracy: ", accuracy)
    print("Detection Rate: ", recall)
    print("F1 Score: ", f1)
    print("ROC AUC Score: ", roc)

    # Predict cho mab
    y_pred_mab = mab.predict(adv_x_test)[0].astype(int).ravel()

    mab_accuracy = accuracy_score(y_true, y_pred_mab)
    mab_recall = recall_score(y_true, y_pred_mab)
    mab_f1 = f1_score(y_true, y_pred_mab)
    mab_roc = roc_auc_score(y_true, y_pred_mab)

    mab_accuracys.append(mab_accuracy)
    mab_drs.append(mab_recall)
    mab_f1s.append(mab_f1)
    mab_rocs.append(mab_roc)

    print("===> MAB:")
    print(classification_report(y_true, y_pred_mab))
    print("Accuracy: ", mab_accuracy)
    print("Detection Rate: ", mab_recall)
    print("F1 Score: ", mab_f1)
    print("ROC AUC Score: ", mab_roc)

# Tổng kết kết quả
print("=====================================")
print("=== DQN ===")
print("Accuracy: ", sum(accuracys) / len(accuracys))
print("Detection Rate: ", sum(drs) / len(drs))
print("F1 Score: ", sum(f1s) / len(f1s))
print("ROC AUC Score: ", sum(rocs) / len(rocs))

print("=== MAB ===")
print("Accuracy: ", sum(mab_accuracys) / len(mab_accuracys))
print("Detection Rate: ", sum(mab_drs) / len(mab_drs))
print("F1 Score: ", sum(mab_f1s) / len(mab_f1s))
print("ROC AUC Score: ", sum(mab_rocs) / len(mab_rocs))

ZOO: 100%|██████████| 500/500 [00:04<00:00, 113.19it/s]


====================> Model:  GaussianNB()
---------- Adversarial data

[🔍] Thống kê số lần mỗi ARM (mô hình) được chọn:
  → Mô hình 0: 427 lần
  → Mô hình 1: 313 lần
  → Mô hình 2: 260 lần
===> DQN:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       500
           1       1.00      1.00      1.00       500

    accuracy                           1.00      1000
   macro avg       1.00      1.00      1.00      1000
weighted avg       1.00      1.00      1.00      1000

Accuracy:  0.998
Detection Rate:  1.0
F1 Score:  0.998003992015968
ROC AUC Score:  0.998
===> MAB:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       500
           1       1.00      1.00      1.00       500

    accuracy                           1.00      1000
   macro avg       1.00      1.00      1.00      1000
weighted avg       1.00      1.00      1.00      1000

Accuracy:  0.998
Detection Rate:  0.996
F1 Scor

ZOO: 100%|██████████| 500/500 [00:03<00:00, 132.62it/s]


====================> Model:  DecisionTreeClassifier()
---------- Adversarial data

[🔍] Thống kê số lần mỗi ARM (mô hình) được chọn:
  → Mô hình 0: 427 lần
  → Mô hình 1: 313 lần
  → Mô hình 2: 260 lần
===> DQN:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       500
           1       1.00      1.00      1.00       500

    accuracy                           1.00      1000
   macro avg       1.00      1.00      1.00      1000
weighted avg       1.00      1.00      1.00      1000

Accuracy:  0.998
Detection Rate:  1.0
F1 Score:  0.998003992015968
ROC AUC Score:  0.998
===> MAB:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       500
           1       1.00      0.99      0.99       500

    accuracy                           0.99      1000
   macro avg       1.00      0.99      0.99      1000
weighted avg       1.00      0.99      0.99      1000

Accuracy:  0.995
Detection Rate:  0

ZOO: 100%|██████████| 500/500 [00:04<00:00, 116.30it/s]


====================> Model:  LogisticRegression()
---------- Adversarial data

[🔍] Thống kê số lần mỗi ARM (mô hình) được chọn:
  → Mô hình 0: 427 lần
  → Mô hình 1: 313 lần
  → Mô hình 2: 260 lần
===> DQN:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       500
           1       1.00      1.00      1.00       500

    accuracy                           1.00      1000
   macro avg       1.00      1.00      1.00      1000
weighted avg       1.00      1.00      1.00      1000

Accuracy:  0.998
Detection Rate:  1.0
F1 Score:  0.998003992015968
ROC AUC Score:  0.998
===> MAB:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       500
           1       1.00      1.00      1.00       500

    accuracy                           1.00      1000
   macro avg       1.00      1.00      1.00      1000
weighted avg       1.00      1.00      1.00      1000

Accuracy:  0.998
Detection Rate:  0.996

ZOO: 100%|██████████| 500/500 [00:21<00:00, 23.08it/s]


====================> Model:  RandomForestClassifier(random_state=42)
---------- Adversarial data

[🔍] Thống kê số lần mỗi ARM (mô hình) được chọn:
  → Mô hình 0: 427 lần
  → Mô hình 1: 313 lần
  → Mô hình 2: 260 lần
===> DQN:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       500
           1       1.00      1.00      1.00       500

    accuracy                           1.00      1000
   macro avg       1.00      1.00      1.00      1000
weighted avg       1.00      1.00      1.00      1000

Accuracy:  0.998
Detection Rate:  1.0
F1 Score:  0.998003992015968
ROC AUC Score:  0.998
===> MAB:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       500
           1       1.00      1.00      1.00       500

    accuracy                           1.00      1000
   macro avg       1.00      1.00      1.00      1000
weighted avg       1.00      1.00      1.00      1000

Accuracy:  0.998
Det

In [14]:
# ===================== Adversarial Attacks on DL Models =====================
models = [mlp_target]
criterions = [criterion_mlp_model_target]
optimizers = [optimizer_mlp_model_target]
input_shapes = [(input_dim,)]
model_types = ['mlp']
accuracys, drs, f1s, rocs = [], [], [], []
mab_accuracys, mab_drs, mab_f1s, mab_rocs = [], [], [], []
attacks_data = attacks_data.astype(np.float32)
normal_data = normal_data.astype(np.float32)
for i, model in enumerate(models):
    classifier = PyTorchClassifier(
        model=model,
        clip_values=(0.0, 1.0),
        loss=criterions[i],
        optimizer=optimizers[i],
        input_shape=input_shapes[i],
        nb_classes=2,
    )

    print(f"\n======> Attacking model: {model_types[i].upper()}")
    
    x_input = attacks_data[:500]
    normal_data_reshaped = normal_data[:500]

    # Sinh mẫu tấn công
    attack = ZooAttack(classifier=classifier, targeted=False, nb_parallel=60)
    x_test_adv = attack.generate(x_input, np.zeros((x_input.shape[0], 1)))

    # Gộp với dữ liệu bình thường
    adv_x_test = np.concatenate((x_test_adv, normal_data_reshaped))
    adv_y_test = np.concatenate((np.ones((x_test_adv.shape[0], 1)), np.zeros((normal_data_reshaped.shape[0], 1))))
    adv_x_test = adv_x_test.astype(np.float64)
    y_true = adv_y_test.astype(int).ravel()
    if adv_x_test.ndim == 3:
        adv_x_test = adv_x_test.reshape((adv_x_test.shape[0], -1))
    print("====================> Model: ", model)
    print("---------- Adversarial data")

    # Predict cho dqn
    y_pred_dqn  = dqn.predict(adv_x_test)[0].astype(int).ravel()

    accuracy = accuracy_score(y_true, y_pred_dqn )
    recall = recall_score(y_true, y_pred_dqn )
    f1 = f1_score(y_true, y_pred_dqn )
    roc = roc_auc_score(y_true, y_pred_dqn )

    accuracys.append(accuracy)
    drs.append(recall)
    f1s.append(f1)
    rocs.append(roc)

    print("===> DQN:")
    print(classification_report(y_true, y_pred_dqn))
    print("Accuracy: ", accuracy)
    print("Detection Rate: ", recall)
    print("F1 Score: ", f1)
    print("ROC AUC Score: ", roc)

    # Predict cho mab
    y_pred_mab = mab.predict(adv_x_test)[0].astype(int).ravel()

    mab_accuracy = accuracy_score(y_true, y_pred_mab)
    mab_recall = recall_score(y_true, y_pred_mab)
    mab_f1 = f1_score(y_true, y_pred_mab)
    mab_roc = roc_auc_score(y_true, y_pred_mab)

    mab_accuracys.append(mab_accuracy)
    mab_drs.append(mab_recall)
    mab_f1s.append(mab_f1)
    mab_rocs.append(mab_roc)

    print("===> MAB:")
    print(classification_report(y_true, y_pred_mab))
    print("Accuracy: ", mab_accuracy)
    print("Detection Rate: ", mab_recall)
    print("F1 Score: ", mab_f1)
    print("ROC AUC Score: ", mab_roc)

# Tổng kết kết quả
print("=====================================")
print("=== DQN ===")
print("Accuracy: ", sum(accuracys) / len(accuracys))
print("Detection Rate: ", sum(drs) / len(drs))
print("F1 Score: ", sum(f1s) / len(f1s))
print("ROC AUC Score: ", sum(rocs) / len(rocs))

print("=== MAB ===")
print("Accuracy: ", sum(mab_accuracys) / len(mab_accuracys))
print("Detection Rate: ", sum(mab_drs) / len(mab_drs))
print("F1 Score: ", sum(mab_f1s) / len(mab_f1s))
print("ROC AUC Score: ", sum(mab_rocs) / len(mab_rocs))


======> Attacking model: MLP


ZOO: 100%|██████████| 500/500 [00:36<00:00, 13.60it/s]


====================> Model:  MLPModel(
  (fc1): Linear(in_features=68, out_features=128, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=64, out_features=2, bias=True)
)
---------- Adversarial data

[🔍] Thống kê số lần mỗi ARM (mô hình) được chọn:
  → Mô hình 0: 427 lần
  → Mô hình 1: 313 lần
  → Mô hình 2: 260 lần
===> DQN:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       500
           1       1.00      1.00      1.00       500

    accuracy                           1.00      1000
   macro avg       1.00      1.00      1.00      1000
weighted avg       1.00      1.00      1.00      1000

Accuracy:  0.998
Detection Rate:  1.0
F1 Score:  0.998003992015968
ROC AUC Score:  0.998
===> MAB:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       500
           1       1.00      0.99      0.99      